In [403]:
import gc
import inspect
import pymysql.cursors


db_name = 'my_orm'
# тут мои данные, так что меняй

In [404]:
def get_connection():
    # тут мои данные, так что меняй
    return pymysql.connect(host='localhost', user='root', password='lumen23291633', db = db_name, cursorclass=pymysql.cursors.DictCursor)

def init_db(host, user, password, db_name):
    connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        db = db_name,
        cursorclass=pymysql.cursors.DictCursor
    )
    try:
        with connection.cursor() as cursor:
            query = 'CREATE DATABASE IF NOT EXISTS `%s`'
            cursor.execute(query, (db_name))
            
        connection.commit()
    
    finally:
        connection.close()
        gc.collect()

In [405]:
# тут мои данные, так что меняй
init_db(
    'localhost',
    'root',
    'lumen23291633',
    db_name
)

/usr/lib/python3.7/site-packages/pymysql/cursors.py:329: Warning: (1007, "Can't create database ''my_orm''; database exists")
  self._do_get_result()


In [406]:
class Validator:
    
    _min_val = 0
    _max_val = 0 
    
    def __init__(self, min_val=0, max_val=0):
        _min_val = min_val
        _max_val = max_val
        
    @classmethod
    def validateMax(cls, value):
        if isinstance(value, int):
            return value if value <= cls._max_val else cls._max_val
        return cls._max_val
    
    @classmethod
    def validateMin(cls, value):
        if isinstance(value, int):
            return value if value >= cls._min_val else cls._min_val
        return cls._min_val
        
    
    
class IntValidator(Validator):
    
    _min_val = -2**31
    _max_val = 2**31 - 1
    
    def __init__(self, min_val=-2**31, max_val=2**31 - 1):
        super().__init__(min_val, max_val)
    
    
class CharValidator(Validator):
    
    _min_val = 1
    _max_val = 8000 
    
    def __init__(self, min_val=1, max_val=8000):
        super().__init__(min_val, max_val)
    
        
class FloatValidator(Validator):
    
    _min_val = -1.79e+308  
    _max_val = 1.79e+308
    
    def __init__(self, min_val=-1.79e+308, max_val=1.79e+308):
        super()._init_(min_val, max_val)
        
        
class Field:
    
    def __init__(self, name='', cls=None, required=True, default=None, primary_key=False):
        self.name = '_' + name
        self.cls = cls
        self.required = required
        self.default = self.validate(default)
        self.primary_key = primary_key
        
    def __get__(self, instance, owner=None):
        if owner is None:
            return AttributeError('This field must be defined in a class!')
        return getattr(instance, self.name, self.default)
    
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        if instance is None:
            raise AttributeError()
        del instance.__dict__[self.name]
        
    def validate(self, value):
        if value is None and not self.required:
            return None
        return self.cls(value)
    
    def cls_validate(self, value):
        if not isinstance(value, self.cls):
            raise AttributeError('The value have to be {0}'.format(self.cls))
        return 
    
    @property
    def is_primary(self):
        return self.primary_key

    @property
    def is_required(self):
        return self.required
    
    @property
    def get_defaule(self):
        return self.default
        
class IntField(Field):
    
    def __init__(self, name='', min_value=None, max_value=None, **kwarg):
        self.min_value = IntValidator.validateMin(min_value)
        self.max_value = IntValidator.validateMax(max_value)
        super().__init__(name, int, **kwarg)
        
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
        
    def __set__(self, instance, value):
        super().cls_validate(value)
       
        if self.min_value > value or self.max_value < value:
            raise AttributeError(
                'Value have to lay between {0} and {1}'.format(
                    self.min_value,
                    self.max_value
                )
            )
        super().__set__(instance, value)

    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class CharField(Field):
    
    def __init__(self, name='', max_length=None, min_length=None, **kwarg):
        self.min_length = CharValidator.validateMin(min_length)
        self.max_length = CharValidator.validateMax(max_length)
        super().__init__(name, str, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().cls_validate(value)
        
        if self.min_length > len(value) or self.max_length < len(value):
            raise AttributeError(
                'Value have to lay between {0} and {1}'.format(
                    self.min_length,
                    self.max_lenght
                )
            )
        super().__set__(instance, value)
        
    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class BoolField(Field):
    
    def __init__(self, name='', **kwarg):
        super().__init__(name, bool, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().__set__(instance, value)
        
    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class FloatField(Field):
    
    def __init__(self, name='', min_val=None, max_val=None, **kwarg):
        self.min_val = CharValidator.validateMin(min_val)
        self.max_val = CharValidator.validateMax(max_val)
        super().__init__(name, float, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().cls_validate(value)
        super().__set__(instance, value)
        
    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class AutoField(IntField):
    
    def __init__(self, name='', **kwarg):
        kwarg['required'] = False
        if kwarg.get('default', None) is not None:
            raise AttributeError(
                'AutoField does not contain default value!'
            )
        super().__init__(name, min_value=0, max_value=2**31-1, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().cls_validate(value)
        super().__set__(instance, value)
    
    def __delete__(self, instance):
        super().__delete__(instance)

In [407]:
class DBTypes:
    
    __types = {
            IntField: 'INT(10)', 
            FloatField: 'FLOAT(53,8)',
            CharField: 'CHAR(255)',
            BoolField: 'BOOLEAN',
            AutoField: 'INT(10)'
    }
    
    __requirement = {False: '', True: 'NOT NULL', None: ''}
    __primary_key = {False: '', True: 'PRIMARY KEY', None: ''}
    
    @classmethod
    def get_db_type(cls, value):
        if isinstance(value, Field):
            return cls.__types[type(value)]
        
    @classmethod
    def get_db_req(cls, value):
        if isinstance(value, Field):
            return cls.__requirement[value.is_required]
        
    @classmethod
    def get_db_prim(cls, value):
        if isinstance(value, Field):
            return cls.__primary_key[value.is_primary]
        
    @classmethod
    def to_db_bool(cls, value):
        if isinstance(value, bool):
            return int(value)
        return value
    
    @classmethod
    def to_db_string(vimcls, value):
        if isinstance(value, str):
            return '"' + value + '"'
        return value

In [408]:
class MetaManager(type):
    
    def __new__(mcs, name, bases, namespace):
        return super().__new__(mcs, name, bases, namespace)
    
class Manager(metaclass=MetaManager):
    
    def __get__(self, instance, owner=None):
        return QuerySet(owner=owner)
    
    def __delete__(self, instance):
        super().__delete__(instance)
        

In [429]:
class QuerySet:
    
    def __new__(cls, **kwargs):
        return super().__new__(cls)
    
    def __init__(self, owner=None, query_set=None):
        if owner is None:
            raise AttributeError('Owner is None')
            
        self.cls = owner
        if query_set is None:
            self.query_set = list()
        else:
            self.query_set = query_set
    
    def create(self, **kwarg):
        return self.cls(**kwarg)
    
    def __len__(self):
        return len(self.query_set)
    
    def __getitem__(self, index):
        return self.query_set[index]
        
    def __setitem__(self, index, value):
        if isinstance(value, self.cls):
            self.query_set[index] = value
            return
        raise AttributeError("Value in not {0}".format(self.cls))
        
    def __delitem__(self, index):
        del self.query_set[index]
        
    def __iter__(self):
        return iter(self.query_set)

    def __reversed__(self):
        return Query_set(self.cls, reversed(self.query_set))
    
    def __str__(self):
        return str(self.query_set)
    
    def append(self, value):
        print(type(value))
        if isinstance(value, self.cls):
            self.query_set.append(value)
            return
        raise AttributeError("Value is not {0}".format(self.cls))
    
    def all(self):
        if len(self):
            return self
        
        connection = get_connection()
        table_name = self.cls.__dict__['Meta'].__dict__['table_name']
        sql_query = 'SELECT * FROM `%s`'
       
        try:
            with connection.cursor() as cursor:
                cursor.execute(sql_query % (table_name))
                self.query_set += [self.cls(**line) for line in cursor.fetchall()]
        finally:
            connection.close()
            gc.collect()
            return self
        
    def __get_local(self, pk=None, **kwargs):
        pass
        
    def get(self, pk=None, **kwargs):
        #if len(self):
        #     return __get_local(self,**kwarg)
            
        table_name = self.cls.__dict__['Meta'].__dict__['table_name']
        sql_query = 'SELECT * FROM `%s` WHERE %s'
        
        if pk is None and not len(kwargs):
            return self
        
        connection = get_connection()
        if isinstance(pk, int) and pk > 0:
            try:
                with connection.cursor() as cursor:
                    name, value = self.cls()._get_primary_name_value()
                    cursor.execute(sql_query % (table_name, '%s=%s' % ('id', pk)))
                    self.query_set += [self.cls(**cursor.fetchone())]
            finally:
                connection.close()
                gc.collect()
                return self
                
        
        fields = self.cls.__dict__['Meta'].__dict__['fields']
        exemp = self.cls()
        where_list = ' AND '.join(
            [
                '%s=%s' % (field_name ,DBTypes.to_db_string(DBTypes.to_db_bool(field_value)))
                for field_name, field_value in kwargs.items() if hasattr(exemp, field_name)
            ]
        )
        print(where_list)
        try:
            with connection.cursor() as cursor:
                cursor.execute(sql_query % (table_name, where_list))
                self.query_set += [self.cls(**line) for line in cursor.fetchall()]
        finally:
            connection.close()
            gc.collect()
            return self
          
        
        
    def delete(self, pk=None, **kwargs):
        
        table_name = self.cls.__dict__['Meta'].__dict__['table_name']
        sql_query = 'DELETE FROM `%s` WHERE %s'
    
        if pk is None and not len(kwargs):
            return 1
        
        connection = get_connection()
        if isinstance(pk, int) and pk > 0:
            try:
                with connection.cursor() as cursor:
                    name, value = self.cls()._get_primary_name_value()
                    cursor.execute(sql_query % (table_name, '%s=%s' % ('id', pk)))
            finally:
                connection.commit()
                connection.close()
                gc.collect()
                return 0
                        
        fields = self.cls.__dict__['Meta'].__dict__['fields']
        exemp = self.cls()
        where_list = ' AND '.join(
            [
                '%s=%s' % (field_name ,DBTypes.to_db_string(DBTypes.to_db_bool(field_value)))
                for field_name, field_value in kwargs.items() if hasattr(exemp, field_name)
            ]
        )
        print(where_list)
        try:
            with connection.cursor() as cursor:
                cursor.execute(sql_query % (table_name, where_list))
        finally:
            connection.commit()
            connection.close()
            gc.collect()
            return self    

In [420]:
class MetaModel(type):
    
    def __new__(mcs, name, bases, namespace):
        super_new = super().__new__
        
        # To choose only bases class constructed by MetaModel
        parents = tuple([b for b in bases if isinstance(b, MetaModel)])
        if not parents:
            return super_new(mcs, name, bases, namespace)
        
        new_namespace = dict()
        module = namespace.pop('__module__', None)
        if module is not None:
            new_namespace['__module__'] = module
            
        classcell = namespace.pop('__classcell__', None)
        if classcell is not None:
            new_namespace['__classcell__'] = classcell

        new_class = super_new(mcs, name, parents, new_namespace)
        
        fields = {name: value for name, value in namespace.items() if isinstance(value, Field)}
        methods = {name: value for name, value in namespace.items() if inspect.isfunction(value)}
        
        fields_name = set(fields.keys())
        
        meta = namespace.pop('Meta', None)
        if meta is not None:
            if not hasattr(meta, 'table_name'):
                setattr(meta, 'table_name', name)
            # There shoulde change conditions
            if not hasattr(meta, 'fields'):
                setattr(meta, 'fields', fields_name)
        else:
            meta = type('Meta', tuple(), {'table_name' : name, 'fields' : fields_name})
        
        # To inherite the fields by MRO
        for base_cls in new_class.mro():
            for field_name, value in base_cls.__dict__.items():  
                if isinstance(value, Field):
                    fields.update({field_name: value})
                    continue
                if inspect.isfunction(value):
                    methods.update({field_name: value})
        
        
        fields = mcs._check_primary(fields)
        meta.fields.update(set(fields.keys()))
        
        fields.update(methods)
        fields.update(mcs._set_manager(namespace))

        
        # To add meta-data
        setattr(new_class, 'Meta', meta)
        
        # To set fields from base classes
        for attr_name, attr_value in fields.items():
            setattr(new_class, attr_name, attr_value)

        return new_class
    
    @classmethod
    def _check_primary(cls, namespace=None):
        if namespace is None:
            return None
        
        primary_dict = {name: value for name, value in namespace.items() if (value.is_primary and value.name != '_id')}
        if len(primary_dict) > 1:
            raise AttributeError(
                'Model has to have only one primary_key field'
            )
        elif len(primary_dict) == 1:
            namespace.pop('id')
            return namespace

        return namespace
    
    @classmethod
    def _set_manager(cls, namespace=None):
        if namespace is None:
            return None
        
        managers = {name: value for name, value in namespace.items() if isinstance(value, Manager)}
        if len(managers) == 0:
            managers.update({'objects': Manager()})
        if len(managers) > 1:
            raise AttributeError(
                'Table have to has only one Manager()'
            )
        return managers
            
        
class Model(metaclass=MetaModel):
    
    class Meta:
        table_name = 'Model'
        fields = {'id'}
    
    id = AutoField('id',primary_key=True)
    objects = Manager()
    
    def __init__(self, **kwargs):
        
        for attr_name, attr_value in kwargs.items():
            if attr_name in self.__class__.__dict__['Meta'].__dict__['fields']:
                setattr(self, attr_name, attr_value)
            
    def _get_primary_name_value(self):
        for name, value in self.__class__.__dict__.items():
            if isinstance(value, Field):
                if value.is_primary:
                    return name, getattr(self, name)
        raise RuntimeError('Model has no primary key!')
        
    def __update(self):
        connection = get_connection()
        table_name = self.__class__.__dict__['Meta'].__dict__['table_name']
        update_reuslt = 0
        
        try:
            with connection.cursor() as cursor:
                name, value = self._get_primary_name_value()
                if value is None:
                    raise RuntimeError('Primary key must be not None')
                if (cursor.execute('SELECT * FROM %s WHERE %s=%s' % (table_name, name, value))):
                    update_query = 'UPDATE {0} SET {1} WHERE {2}'
                    attr_list = [
                        str(attr_name) + '=' + str(DBTypes.to_db_string(DBTypes.to_db_bool(getattr(self, attr_name))))
                        for attr_name, attr_value in self.__class__.__dict__.items()
                        if isinstance(attr_value, Field) and not attr_value.is_primary
                    ]
                    attr_list = ','.join(attr_list)
                    update_query = update_query.format(
                        table_name,
                        attr_list,
                        '%s=%s' % (name, value)
                    )
                    cursor.execute(update_query)
                    connection.commit()
                    update_reuslt = 1
                    
        finally:
            connection.close()
            gc.collect()
            return update_reuslt
                         
                
    def save(self):
        connection = get_connection()
        table_name = self.__class__.__dict__['Meta'].__dict__['table_name']
        
        if self.__update():
            return
        
        try:
            with connection.cursor() as cursor:
                value_list = list()
                name_list = list()
                string_list = list()
                for field_name in self.__class__.__dict__['Meta'].__dict__['fields']:
                    value_list.append(getattr(self, field_name))
                    name_list.append(field_name)
                    string_list.append(
                        '`%s` %s %s %s' % (
                            field_name, 
                            DBTypes.get_db_type(self.__class__.__dict__[field_name]),
                            DBTypes.get_db_req(self.__class__.__dict__[field_name]),
                            DBTypes.get_db_prim(self.__class__.__dict__[field_name]),
                        )
                    )
                string_list = ','.join(string_list)
                cursor.execute(
                    'CREATE TABLE IF NOT EXISTS `%s` (%s)' % (
                        table_name,
                        string_list
                    )
                )
                value_list = [str(DBTypes.to_db_string(DBTypes.to_db_bool(e))) for e in value_list]
                value_list = ','.join(value_list)
                name_list = ','.join(name_list)
                query_insert = 'INSERT INTO %s (%s) VALUES (%s)' % (
                    table_name,
                    name_list,
                    value_list
                )
                cursor.execute(query_insert)
                connection.commit()
        finally:
            connection.close()
            gc.collect()

In [430]:
class User(Model):
    
    TelNumber = IntField('TelNumber',required=True, default=2)
    Name =      CharField('Name',required=True, default='fdf')
    Sex =       BoolField('Sex',required=True, default=True)
    Money  =    FloatField('Money',required=False, default=0.0)
    
    def __init__(self, **kwarg):
        super().__init__(**kwarg)
    
    def update_money(self, value):
        self.Money = value
        
    

class Stuff(User):
    Post = IntField(required=True, default = -1)
    
    


0


'SELECT * FROM `asd` WHERE id=1'